In [3]:
import pygame
from math import dist
from queue import PriorityQueue
import math

WIDTH = 500
WIN = pygame.display.set_mode((WIDTH, WIDTH))  # dimension to make it a square
pygame.display.set_caption("A* Path Finding Algorithm")

RED = (235, 77, 75)
GREEN = (186, 220, 88)
BLUE = (48, 51, 107)
YELLOW = (249, 202, 36)
WHITE = (255, 255, 255)
BLACK = (53, 59, 72)
PURPLE = (130, 88, 159)
ORANGE = (225, 95, 65)
GREY = (128, 128, 128)
TURQUOISE = (10, 189, 227)

class Send_spot:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    

class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_barrier(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == TURQUOISE

    def reset(self):
        self.color = WHITE

    def make_start(self):
        self.color = ORANGE

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK

    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE

    def draw(self, win):
        pygame.draw.rect(
            win, self.color, (self.x, self.y, self.width, self.width))
        
    def update_neighbors(self, grid):
        self.neighbors = []
        # DOWN
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():  # UP
            self.neighbors.append(grid[self.row - 1][self.col])

        # RIGHT
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier():  # LEFT
            self.neighbors.append(grid[self.row][self.col - 1])

        # Above the right diagonal
        if self.row > 0 and self.col < self.total_rows - 1 and not grid[self.row - 1][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col + 1])
        
        # Down the right diagonal
        if self.row < self.total_rows - 1 and self.col < self.total_rows - 1 and not grid[self.row + 1][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col + 1])
                
        # Above the left diagonal
        if self.row > 0 and self.col > 0 and not grid[self.row - 1][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col - 1])
            
        # Down the left diagonal
        if self.row < self.total_rows - 1 and self.col > 0 and not grid[self.row + 1][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col - 1])
            
    def __lt__(self, other):
        return False

def heuristic(p1, p2):
    return dist(p1, p2)

def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

def calculate_angle(prev_point,current,next_point):
    if prev_point == None:
        #첫 실행 시 이전 노드가 None이다 따라서 이때는 각을 계산하지 않고 0을 반환한다
        return 0
    prev_x,prev_y = prev_point.x,prev_point.y
    current_x,current_y = current.row,current.col
    next_x = next_point.x
    next_y = next_point.y
    #print("cal angle",prev_x,prev_y,current_x,current_y,next_x,next_y)
    d1 = (current_x - prev_x, current_y - prev_y)
    d2 = (next_x - current_x, next_y - current_y)

    cos_angle = (d1[0]*d2[0] + d1[1]*d2[1]) / (math.hypot(*d1) * math.hypot(*d2))
    angle = math.degrees(math.acos(cos_angle))
    #print("angle",angle)
    return angle


def assign_weight(prev_point,current,next_point):
    angle = int(calculate_angle(prev_point,current,next_point))
    #이전 노드, 현재 노드, 다음 노드를 통해 각을 계산한다
    if angle == 45:
        #백터 계산에 의해 값이 45도가 나오면 이동한 벡터의 사잇값이 135도가 된다
        weight = 2
    elif angle == 90:
        weight = 3
    elif angle == 0:
        weight = 0
    elif angle == 135:
        #벡터 계산에 의해 값이 135도가 나오면 벡터의 사잇값이 45도가 된다 따라서 가장 가중치의 값이 크다
        weight = 9
    return weight

def new_astar(start,prev_point,current,store_f,visited):
    min_fscore = 100
    #최소 fscore를 100으로 초기화 한다
    min_spot = None
    #최적 위치를 None으로 초기화 한다
    x_weight = [0,1,1,1,0,-1,-1,-1]
    #상하좌우 대각선 이동을 위한 위치 가중치
    y_weight = [-1,-1,0,1,1,1,0,-1]
    #상하좌우 대각선 이동을 위한 위치 가중치
    
    for i in range(8):
        #상하좌우 대각선이기 때문에 range(8)
        current_x = current.row
        #현재 위치의 열값을 추출한다
        current_y = current.col
        #현재 위치의 행 값을 추출한다
        weight_x = current_x + x_weight[i]
        #위치 이동 가중치를 더한다
        weight_y = current_y + y_weight[i]
        #위치 이동 가중치를 더한다

        if weight_x == start.row and weight_y == start.col:
            #만약 이동한 위치가 시작점 즉 역추척의 종착점이라면
            return 0
            #0을 반환한다
            
        for x in store_f:
            #store_f 리스트에 있는 요소들을 모두 불러온다
            f_score,n_x,n_y = x
            if weight_x == n_x and weight_y == n_y and (n_x,n_y) not in visited:
                #만약 이동한 위치가 f_score리스트 안에 존재하고 이미 방문한 노드가 아니라면
                next_point = Send_spot(weight_x,weight_y)
                #이동한 위치를 send_spot 클래스 자료형을 변환한다
                angle_weight = assign_weight(prev_point,current,next_point)
                #각 가중치 값을 구한다
                new_fscore = f_score + angle_weight
                #각 가중치 값을 기존 f_score에 더한다
                #print("fscore:",n_x,n_y,new_fscore)
                if new_fscore < min_fscore:
                    #만약 구한 f_score가 최소 f_score보다 작다면
                    min_fscore = new_fscore
                    #최소 f_score를 업데이트 하고
                    min_spot = Send_spot(weight_x,weight_y)
                    #최적 이동 위치를 Send_spot 클래스 형태로 업데이트 한다
    #print("minspot",min_spot.x,min_spot.y)
    return min_spot
    #최적 이동 위치를 반환한다
                    
                
                
                
def find_path(start,current,store_f,draw,grid):
    visited = set()
    #방문한 노드를 저장할 집합
    prev_point = None
    #함수를 처음 호출 할 시 이전 지점이 없기 때문에 None으로 초기화 한다
    while 1:
        
        best = new_astar(start,prev_point,current,store_f,visited)
        #최적위치를 구한다
        if best == 0:
            #최적 위치가 시작점이라면 반복문을 종료한다
            break
        prev_point = Send_spot(current.row,current.col)
        #이전 위치를 현재위치로 업데이트하고
        #print("prev",prev_point.x,prev_point.y)
        visited.add((current.row,current.col))
        #방문노드 집합에 현재 노드를 추가한다
        current.row,current.col = best.x,best.y
        #현재 노드를 탐색한 최적위치로 업데이트 한다
        spot_object = grid[current.row][current.col]
        #grid를 사용하여 spot_object에 현위치를 노드화 시킨다 
        #탐색 알고리즘에서는 편의상 send_spot이라는 클래스로 인자를 전달하는데
        #그 자료형을 grid를 사용하여 spot화 시키는 것이다
        spot_object.make_path()
        draw()
    
    
def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    store_f = []
    #f_score와 좌표들을 저장할 리스트를 선언합니다
    # 시작 노드에서 이 노드까지의 현재 최단 거리를 추적합니다
    # 초기화
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    
    # 이 노드에서 엔드 노드까지의 예측 거리를 추적합니다
    # 초기화
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = heuristic(start.get_pos(), end.get_pos())

    open_set_hash = {start}

    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            #reconstruct_path(came_from, end, draw)
            find_path(start,current,store_f,draw,grid)
            #find_path 함수를 호출합니다 
            end.make_end()
            return True

        for neighbor in current.neighbors:
            if current.row != neighbor.row and current.col != neighbor.col:
                temp_g_score = g_score[current] + 1.4
            else:
                temp_g_score = g_score[current] + 1.0

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + heuristic(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()
                    
        draw()
        #print(current.row,current.col)
        if current != start:
            current.make_closed()
            store_f.append((f_score[current],current.row,current.col))
            #fscore와 열,행을 저장합니다
            
    return False


def make_grid(rows, width):
    grid = []
    gap = width // rows  # integer division: gap b/w each of these rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows)
            grid[i].append(spot)

    return grid


def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap),
                         (width, i * gap))  # horizontal line
        for j in range(rows):
            pygame.draw.line(win, GREY, (j * gap, 0),
                             (j * gap, width))  # vertical lines


def draw(win, grid, rows, width):
    win.fill(WHITE)

    for row in grid:
        for spot in row:
            spot.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()


def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos

    row = y // gap
    col = x // gap

    return row, col

def main(win, width):
    ROWS = 10
    grid = make_grid(ROWS, width)

    start = None
    end = None

    run = True
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT MOUSE BUTTON: 0
                pos = pygame.mouse.get_pos()
                # actual spot in 2D list where mouse is clicked
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]

                # if start and end aren't done
                if not start and spot != end:
                    start = spot
                    start.make_start()

                    # to avoid overlapping of start and end node
                elif not end and spot != start:
                    end = spot
                    end.make_end()

                elif spot != end and spot != start:
                    spot.make_barrier()

            elif pygame.mouse.get_pressed()[2]:  # RIGHT MOUSE BUTTON: 2
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)

                    algorithm(lambda: draw(win, grid, ROWS, width),
                              grid, start, end)

                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)
     
    pygame.quit()

main(WIN, WIDTH)